# EDA

In [1]:
import pandas as pd
import numpy as np
import datetime

In [121]:
df = pd.read_csv("data/preprocessed_data.csv")

sử dụng EDA của Tùng để bỏ bớt 1 số trường có thể map 1-1 với các trường khác, hoặc các trường có ít correlation với giá 

In [122]:
df.drop(
    [
        'url',
        'date',
        'out_color',
        'in_color',
        'num_door',
        'num_seat',
        'gear_type',
        'drive_type',
        'cylinder_capacity'
    ],
    axis=1,
    inplace=True
)

In [123]:
df

,brand,serie,model_year,price_vnd,origin,status,category,used_distance
0,Ford,Focus,2018.0,478,Lắp ráp trong nước,Xe đã dùng,Sedan,0
1,Mazda,CX5,2021.0,999,Lắp ráp trong nước,Xe mới,SUV,0
2,Kia,Cerato,2018.0,510,Lắp ráp trong nước,Xe đã dùng,Sedan,2
3,Isuzu,Dmax,2021.0,580,Nhập khẩu,Xe mới,Bán tải / Pickup,0
4,Kia,Sorento,2021.0,1230,Lắp ráp trong nước,Xe đã dùng,SUV,0
...,...,...,...,...,...,...,...,...
41608,Hãng khác,NaN,NaN,65,Lắp ráp trong nước,Xe đã dùng,Truck,0
41609,Toyota,Corolla,2002.0,155,Lắp ráp trong nước,Xe đã dùng,Sedan,0
41610,Ford,Ranger,2016.0,675,Nhập khẩu,Xe đã dùng,Bán tải / Pickup,0
41611,Chrysler,300C,2008.0,555,Nhập khẩu,Xe đã dùng,Sedan,58000


In [124]:
df['age'] = datetime.datetime.now().year - df['model_year']

### Phân tích ban đầu

Các thông tin có thể validate:
- used_distance: Dựa vào số km/năm (vd: ko thể chạy quá 15v km/năm)
- status: đối chiếu với used_distance, xe mới thì used_distance phải = 0, hoặc nếu xe đã dùng mà used_distance = 0 thì phải xem xét sửa used_distance thành trung bình km/năm x số năm.

Các thông tin ko thể validate:
- model_year/năm
- brand, series


## validate used_distance

- Nhận định: với các xe > 15v/năm, điền bừa, hoặc model_year bị sai -> lọc bỏ

In [125]:
df_valid_used_distance = df[df.used_distance/df.age < 15000]

In [126]:
df[df.used_distance/df.age > 150000]

,brand,serie,model_year,price_vnd,origin,status,category,used_distance,age
2402,Ford,Ranger,2016.0,760,Nhập khẩu,Xe đã dùng,Bán tải / Pickup,1000000,6.0
2729,Toyota,Innova,2009.0,265,Lắp ráp trong nước,Xe đã dùng,Crossover,2031509,13.0
5007,Kia,Soluto,2019.0,350,Lắp ráp trong nước,Xe đã dùng,Sedan,40000000,3.0
8603,Kia,Morning,2016.0,220,Lắp ráp trong nước,Xe đã dùng,Hatchback,6600000,6.0
9724,Ford,Everest,2022.0,1190,Nhập khẩu,Xe mới,SUV,1086,0.0
...,...,...,...,...,...,...,...,...,...
39855,Mitsubishi,Triton,2019.0,785,Nhập khẩu,Xe đã dùng,Bán tải / Pickup,2600000,3.0
39983,Kia,Cerato,2016.0,465,Lắp ráp trong nước,Xe đã dùng,Sedan,1000000,6.0
40152,Hyundai,i10,2019.0,280,Lắp ráp trong nước,Xe đã dùng,Sedan,650000,3.0
40198,Toyota,Land Cruiser,1997.0,200,Nhập khẩu,Xe đã dùng,SUV,9999999,25.0


#### Phân tích used_distance của xe đã dùng

validate xe đã dùng: 12297/31086 là ko có dữ liệu used_distance (= 0)
- approach 1: sửa lại toàn bộ sử dụng used_distance TB theo age
- approach 2: với các xe age < 3 năm, để yên và chuyển thành xe mới (nhầm status). các xe >= 3 năm sửa lại thành TB theo age.

In [127]:
df_used = df[df.status == "Xe đã dùng"]
df_used_0_distance = df_used[df_used.used_distance == 0]
print(f"{len(df_used_0_distance)}/{len(df_used)}")

12297/31086


Sử dụng xe đã dùng với used_distance>0 để xây dựng used_distance trung bình theo age. Mục đích để sau lấy giá trị điền vào các hàng mà used_distance bị sai

In [128]:
df_used_car = df_valid_used_distance[df_valid_used_distance["status"] == "Xe đã dùng"]
df_valid_used_distance = df_valid_used_distance.drop(df_used_car[df_used_car.used_distance == 0].index)

In [129]:
df_mean_distance_by_age = df_valid_used_distance.groupby(["age"]).mean()
df_mean_distance_by_age["used_distance"] = df_mean_distance_by_age["used_distance"].astype("int")
df_mean_distance_by_age

,model_year,price_vnd,used_distance
age,,,
1.0,2021.0,1795.695614,310
2.0,2020.0,1737.674692,9926
3.0,2019.0,1199.125169,23383
4.0,2018.0,1086.413697,33782
5.0,2017.0,999.474932,44948
6.0,2016.0,1112.166572,53629
7.0,2015.0,941.352236,60080
8.0,2014.0,1025.280682,62941
9.0,2013.0,773.799302,70324


In [130]:
# approach 1: sửa tất cả 
def lambda_func(x):
    if x.status == "Xe đã dùng" and x.used_distance == 0:
        if x["age"]>0: 
            return df_mean_distance_by_age.loc[x["age"]].used_distance
        else:
            return x.used_distance
    else:
        return x.used_distance

In [131]:
df

,brand,serie,model_year,price_vnd,origin,status,category,used_distance,age
0,Ford,Focus,2018.0,478,Lắp ráp trong nước,Xe đã dùng,Sedan,0,4.0
1,Mazda,CX5,2021.0,999,Lắp ráp trong nước,Xe mới,SUV,0,1.0
2,Kia,Cerato,2018.0,510,Lắp ráp trong nước,Xe đã dùng,Sedan,2,4.0
3,Isuzu,Dmax,2021.0,580,Nhập khẩu,Xe mới,Bán tải / Pickup,0,1.0
4,Kia,Sorento,2021.0,1230,Lắp ráp trong nước,Xe đã dùng,SUV,0,1.0
...,...,...,...,...,...,...,...,...,...
41608,Hãng khác,NaN,NaN,65,Lắp ráp trong nước,Xe đã dùng,Truck,0,NaN
41609,Toyota,Corolla,2002.0,155,Lắp ráp trong nước,Xe đã dùng,Sedan,0,20.0
41610,Ford,Ranger,2016.0,675,Nhập khẩu,Xe đã dùng,Bán tải / Pickup,0,6.0
41611,Chrysler,300C,2008.0,555,Nhập khẩu,Xe đã dùng,Sedan,58000,14.0


In [132]:
df["clean_used_distance"] = df.apply(lambda x: lambda_func(x), axis=1)

In [133]:
df_new_car = df[df.status == "Xe mới"]
index_wrong_status = df_new_car[df_new_car.used_distance > 0].index
df.at[index_wrong_status,"status"] = "Xe đã dùng"

In [134]:
cat_columns_df = df[['brand', 'serie', 'origin', 'status', 'category']]
numeric_columns_df = df[['price_vnd', 'used_distance', 'age']]

In [135]:
df = pd.concat([numeric_columns_df, pd.get_dummies(cat_columns_df)], axis=1)

In [138]:
df = df.dropna()

# ML

In [139]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['price_vnd'], axis=1), 
    df['price_vnd'],
    test_size=0.2
)

In [140]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [141]:
model.fit(X_train, y_train)

LinearRegression()

# Evaluation

In [142]:
from helpers.evaluation import get_summary_result

In [143]:
get_summary_result(model, X_test, y_test)

{'Model': 'LinearRegression', 'Hyperparams': "{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False, 'positive': False}", 'R2': 0.7761625727136361, 'RMSE': 897.804497209276, 'MAE': 313.43030616121416, 'MAPE': 2.812415671706137, 'Time': '22:48:45 26-03-2022'}
